In [ ]:
# Models
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from prophet import Prophet
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Mahine Learning Tools
from pmdarima import auto_arima
from sklearn.metrics import  mean_squared_error 
from sklearn.model_selection import TimeSeriesSplit

# Data Manipulation Tools
import pandas as pd
import numpy as np

# Data Visualization Tools
import matplotlib.pyplot as plt
import seaborn as sns

# Data Preprocessing Tools
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# Import the Dataset
priceData = pd.read_csv("./data/processed/pricesList.csv")
rainfallData = pd.read_csv("./data/processed/rainfallData.csv")

In [ ]:
# set the index to the date
def extractDate(data):
    data2 = data.copy()
    data2["Datetime"] = pd.to_datetime(data2[['Year', 'Month']].assign(day=(data2["Week"]-1)*7 +1))
    data2 = data2.set_index('Datetime')
    return data2
priceData = extractDate(priceData)

## Visualize Dataset

In [ ]:
# Get all the unique values of the Name column in the dataset and iterate through them to display the price trend
uniqueNames = priceData['Name'].unique()
# fig with subplots for each name
fig, axs = plt.subplots(len(uniqueNames), 1, figsize=(20, 15))
for i, name in enumerate(uniqueNames):
    # filter the dataset to get the data for the current name
    priceData[priceData['Name'] == name]["Price"].plot(ax=axs[i], label=name)
    axs[i].legend(loc='upper left')
plt.show()

In [ ]:
# Visualize one of the crops
plt.figure(figsize=(15, 7))
priceData[priceData['Name'] == uniqueNames[0]]["Price"].loc["2020-01-01":"2023-12-15"].copy().dropna().plot()
plt.title(uniqueNames[0])
plt.show()

In [ ]:

# fig with one plot for all names
fig, ax = plt.subplots(1, 1, figsize=(20, 15))
for i, name in enumerate(uniqueNames):
    # filter the dataset to get the data for the current name from date 2016-01-01 to 2019-12-31
    priceData[priceData['Name'] == name]["Price"].loc['2021-01-01':'2024-12-31'].plot(ax=ax, label=name)
    ax.legend(loc='upper left')
plt.title("Abnormal Price Trend in 2023 to 2024")
plt.show()

## Extract Data

In [ ]:
# create a new dataset for processing
df = priceData.copy()
# extract only one vegetable for the prediction
vegetable = uniqueNames[0]
df = df[df["Name"] == vegetable]
df.drop(columns=["Name"], inplace=True)

In [ ]:
# merge the rainfall data with the price data
for index, row in rainfallData.iterrows():
    year = row['year']
    month = row['month']
    df.loc[(df['Year'] == year) & (df['Month'] == month), 'anuradhapura'] = row['anuradhapura']
    df.loc[(df['Year'] == year) & (df['Month'] == month), 'jaffna'] = row['jaffna']
    df.loc[(df['Year'] == year) & (df['Month'] == month), 'nuwaraeliya'] = row['nuwaraeliya']


## Simple Imputer

In [ ]:
# Simple Imputer for missing values in the dataset
def imputeData(data, column):
    imputer = SimpleImputer(missing_values = pd.NA, strategy ='mean')
    imputer.fit(data[[column]])
    # round to 2 decimal places
    data[column] = imputer.transform(data[[column]])
    data[column] = data[column].round(2)

imputeData(df, "Price")
imputeData(df, "anuradhapura")
imputeData(df, "jaffna")
imputeData(df, "nuwaraeliya")

# drop the columns where price is null
df = df.dropna(subset=["Price"])

## Lag Features

In [ ]:
def createLagFeatures(data, lag):
    data2 = data.copy()
    data2["Price_Lag"] = data2["Price"].shift(lag)
    data2["anuradhapura_Lag"] = data2["anuradhapura"].shift(lag)
    data2["jaffna_Lag"] = data2["jaffna"].shift(lag)
    data2["nuwaraeliya_Lag"] = data2["nuwaraeliya"].shift(lag)
    return data2
df = createLagFeatures(df, 1)

## Rolling Features

In [ ]:
def createRollingFeatures(data, window):
    data2 = data.copy()
    data2["Price_Mean"] = data2["Price"].rolling(window=window).mean().shift(1)
    data2["anuradhapura_Mean"] = data2["anuradhapura"].rolling(window=window).mean().shift(1)
    data2["jaffna_Mean"] = data2["jaffna"].rolling(window=window).mean().shift(1)
    data2["nuwaraeliya_Mean"] = data2["nuwaraeliya"].rolling(window=window).mean().shift(1)
    return data2
df = createRollingFeatures(df, 3)


In [ ]:
# Remove null values
df.dropna(inplace=True)

## Data Visualization

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=(15, 5))
df["Price"].plot(ax=ax, label="After Imputation", color="blue")
priceData[priceData["Name"] == vegetable]["Price"].plot(ax=ax, label="Before Imputation", color="red")
plt.legend(loc='upper left')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 9))
for column in df.select_dtypes(include=['object']).columns:
    label_encoder = LabelEncoder()
    df[column] = label_encoder.fit_transform(df[column])
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', ax=ax)
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
df.head()

## Preprocess Data

In [ ]:
split_date = "2022-08-01"
train = df.loc[df.index <= split_date]
test = df.loc[df.index > split_date]
fix, ax = plt.subplots(1, 1, figsize=(15, 5))
train['Price'].plot(ax=ax, label="Train Set")
test['Price'].plot(ax=ax, label="Test Set")
ax.axvline(split_date, color="black", ls="--")
ax.legend("Training set", "Test Set")
plt.legend(loc='upper left')
plt.show()

In [ ]:
# divide the dataset into features and target
FEATURES = ["Week", "Month", "Year",
            #"anuradhapura", "jaffna", "nuwaraeliya", 
            "Price_Lag", # "anuradhapura_Lag", "jaffna_Lag", "nuwaraeliya_Lag",
            "Price_Mean", # "anuradhapura_Mean", "jaffna_Mean", "nuwaraeliya_Mean"
        ]
TARGET = "Price"

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

## Model Training & Evaluation

### Facebook Prophet

In [ ]:
# Train the model
fb_prophet = Prophet()
fb_prophet.fit(train.reset_index().rename(columns={"Datetime": "ds", "Price": "y"}))

# Evaluate the model
prophet_pred = fb_prophet.predict(test.reset_index().rename(columns={"Datetime": "ds", "Price": "y"}))
fix, ax = plt.subplots(figsize=(10, 5))
fig = fb_prophet.plot(prophet_pred, ax=ax)
test["Price"].plot(ax=ax, label="True Price", color="r")
plt.legend(loc='upper left')
plt.show()

### XGB Regressor

In [ ]:
# Model Training
XGB_reg = XGBRegressor(n_estimators=2000, learning_rate=0.01)
XGB_reg.fit(X_train, y_train, early_stopping_rounds=2000, eval_set=[(X_test, y_test)], verbose=False)

# Evaluate the model
XGB_pred = XGB_reg.predict(X_test)
mse = mean_squared_error(y_test, XGB_pred)
score = XGB_reg.score(X_test, y_test)
print("MSE: ", mse)
print("Score: ", score*100)

fix, ax = plt.subplots(1, 1, figsize=(15, 5))
y_test.plot(ax=ax, label="Actual Price", color="red")
pd.Series(XGB_pred, index=y_test.index).plot(ax=ax, label="Predicted Price", color="blue")
plt.legend(loc='upper left')
plt.show()

### Random Forest Regressor

In [ ]:
RF_reg = RandomForestRegressor(bootstrap=True, criterion='absolute_error',n_estimators=5000, random_state=None, verbose=False)
RF_reg.fit(X_train, y_train)

# Evaluate the model
RF_pred = RF_reg.predict(X_test)
mse = mean_squared_error(y_test, RF_pred)
score = RF_reg.score(X_test, y_test)
print("MSE: ", mse)
print("Score: ", score*100)

fix, ax = plt.subplots(1, 1, figsize=(15, 5))
y_test.plot(ax=ax, label="Actual Price", color="red")
pd.Series(RF_pred, index=y_test.index).plot(ax=ax, label="Predicted Price", color="blue")
plt.legend(loc='upper left')
plt.show()

### Cat Boost Regressor

In [ ]:
CB_reg =  CatBoostRegressor(iterations=100000, depth=1, learning_rate=0.001, loss_function='RMSE')
CB_reg.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

# Evaluate the model
CB_pred = CB_reg.predict(X_test)
mse = mean_squared_error(y_test, CB_pred)
score = CB_reg.score(X_test, y_test)
print("MSE: ", mse)
print("Score: ", score*100)

fix, ax = plt.subplots(1, 1, figsize=(15, 5))
y_test.plot(ax=ax, label="Actual Price", color="red")
pd.Series(CB_pred, index=y_test.index).plot(ax=ax, label="Predicted Price", color="blue")
plt.legend(loc='upper left')
plt.show()

### ARIMA Model

In [ ]:
# find the best model
stepwise_fit = auto_arima(df["Price"], exogenous=train, seasonal=True, m=12, stepwise=True, error_action="ignore",max_p=3, max_q=3, max_P=3, max_Q=3,)
# Train the model
arima = SARIMAX(train[TARGET],exog=train[FEATURES], order=(stepwise_fit.get_params()["order"]),seasonal_order=(0, 0, 0, 0))
arima = arima.fit(maxiter=500, disp=False)
# Evaluate the model
start = len(train)
end = len(train) + len(test) - 1
arima_pred = arima.predict(start=start, end=end, typ="levels", exog=test[FEATURES]).rename("ARIMA Predictions")
arima_pred.index = test.index
fix, ax = plt.subplots(figsize=(10, 5))
test["Price"].plot(ax=ax, label="True Price", color="red")
arima_pred.plot(ax=ax, label="ARIMA Predictions", color="blue")
plt.legend(loc='upper left')
plt.show()


### All Model Evaluation

In [ ]:
# Foreacast the price
prediction = test.copy()
prediction["XGB"] = XGB_pred
prediction["RF"] = RF_pred
prediction["CB"] = CB_pred
prediction["Prophet"] = prophet_pred["yhat"].values
prediction["ARIMA"] = arima_pred
ax = prediction["Price"].plot(label="Actual Price", figsize=(15, 5), color="red")
prediction["XGB"].plot(ax=ax, label="XG Boost", style="--", color="purple")
prediction["RF"].plot(ax=ax, label="Random Forest", style="--", color="blue")
prediction["CB"].plot(ax=ax, label="Cat Boost", style="--", color="cyan")
prediction["ARIMA"].plot(ax=ax, label="ARIMA", style="--", color="orange")
prediction["Prophet"].plot(ax=ax, label="Prophet", style="--", color="green")
plt.legend(loc='upper left')
plt.show()

## Time Series Cross Validation

In [ ]:
# Time Series Cross Validation
splits = 2
tss = TimeSeriesSplit(n_splits=splits, test_size=5,gap=-1)
df2 = df.copy().sort_index()
fig, axs = plt.subplots(splits, 1, figsize=(15, 15))
fold = 0

XGB_mse, RF_mse, CB_mse, ARIMA_mse = [], [], [], []
XGB_rmse, RF_rmse, CB_rmse, ARIMA_rmse = [], [], [], []

for train_idx, test_idx in tss.split(df2):
    print("Prediction set " + str(fold+1) + " started")
    train_tss = df2.iloc[train_idx]
    test_tss = df2.iloc[test_idx]
    X_tss_train = train_tss[FEATURES]
    y_tss_train = train_tss[TARGET]
    X_tss_test = test_tss[FEATURES]
    y_tss_test = test_tss[TARGET]
    # plot train and test set
    train_tss["Price"].loc["2023-01-01":].plot(ax=axs[fold], label="Train Set", color="blue")
    test_tss["Price"].plot(ax=axs[fold], label="Test Set", color="red")

    # Train the models
    XGB_reg = XGB_reg.fit(X_tss_train, y_tss_train)
    RF_reg = RF_reg.fit(X_tss_train, y_tss_train)
    CB_reg = CB_reg.fit(X_tss_train, y_tss_train)
    arima = SARIMAX(train_tss[TARGET],exog=train_tss[FEATURES], order=(stepwise_fit.get_params()["order"]),seasonal_order=(0, 0, 0, 0))
    arima = arima.fit(maxiter=500, disp=False)

    # Evaluate the models
    XGB_pred = XGB_reg.predict(X_tss_test)
    RF_pred = RF_reg.predict(X_tss_test)
    CB_pred = CB_reg.predict(X_tss_test)
    arima_pred = arima.predict(start=len(train_tss), end=len(train_tss) + len(test_tss) - 1, typ="levels", exog=test_tss[FEATURES]).rename("ARIMA Predictions")
    arima_pred.index = test_tss.index

    # plot the predictions
    pd.Series(XGB_pred, index=y_tss_test.index).plot(ax=axs[fold], label="XG Boost", color="green", style="--")
    pd.Series(RF_pred, index=y_tss_test.index).plot(ax=axs[fold], label="Random Forest", color="blue", style="--")
    pd.Series(CB_pred, index=y_tss_test.index).plot(ax=axs[fold], label="Cat Boost", color="orange", style="--")
    arima_pred.plot(ax=axs[fold], label="ARIMA", color="cyan", style="--")
    axs[fold].legend(loc='upper left')

    XGB_mse.append(mean_squared_error(y_tss_test, XGB_pred))
    RF_mse.append(mean_squared_error(y_tss_test, RF_pred))
    CB_mse.append(mean_squared_error(y_tss_test, CB_pred))
    ARIMA_mse.append(mean_squared_error(y_tss_test, arima_pred))

    # RMSE
    XGB_rmse.append(np.sqrt(mean_squared_error(y_tss_test, XGB_pred)))
    RF_rmse.append(np.sqrt(mean_squared_error(y_tss_test, RF_pred)))
    CB_rmse.append(np.sqrt(mean_squared_error(y_tss_test, CB_pred)))
    ARIMA_rmse.append(np.sqrt(mean_squared_error(y_tss_test, arima_pred)))

    fold += 1

print("RMSE Values: ")

print("XG Boost: ", np.mean(XGB_rmse))
print("Random Forest: ", np.mean(RF_rmse))
print("Cat Boost: ", np.mean(CB_rmse))
print("ARIMA: ", np.mean(ARIMA_rmse))

print("MSE Values: ")
print("XG Boost: ", np.mean(XGB_mse))
print("Random Forest: ", np.mean(RF_mse))
print("Cat Boost: ", np.mean(CB_mse))
print("ARIMA: ", np.mean(ARIMA_mse))

In [ ]:
# save the model
import pickle
filename = './models/finalized_model.sav'

# train the catboost model on the entire dataset
CB_reg = CB_reg.fit(df[FEATURES], df[TARGET])

pickle.dump(CB_reg, open(filename, 'wb'))